In [ ]:
import numpy as np
import math, cv2
from matplotlib import image
import scipy.stats as st
import matplotlib.pyplot as plt

def init(img, k):
    clusterProbs = np.random.rand(k)
    clusterProbs = clusterProbs / np.sum(clusterProbs)

#     c_x = np.random.rand(k)*100
#     c_y = np.random.rand(k)*100
#     c_z = np.random.rand(k)
#     clusterMeans = np.array(list(zip(c_x, c_y, c_z)), dtype=np.float32)

    _, clusterMeans = kMeans(img, k)

    clusterCovariances = np.array([np.identity(3)*100] * k)

    return clusterProbs, clusterMeans, clusterCovariances

def expectation(img, clusterProbs, clusterMeans, clusterCovariences, k):
    n, d = img.shape

#   for each point find the probability that it belongs to each cluster
#   by multiplying the cluster's probability (initial assignment is random)
#   and the the multivariate Gaussian of the point
    pointProbsK = np.array([np.zeros(n)]*k)
    
    for i in range(k):
        multivariate_normal = st.multivariate_normal.pdf(img, clusterMeans[i], clusterCovariences[i])
        pointProbsK[i] = clusterProbs[i]*multivariate_normal
    
    sum_pointProbsK = np.sum(pointProbsK, axis=0)
    pointProbsK = pointProbsK / sum_pointProbsK
    return pointProbsK

def maximization(img, pointProbsK, k):
    n, d = img.shape
    
#   for each cluster calculate the weight and probability of the cluster
#   by adding and averaging (in that order) the probability of each point belonging to the cluster
    totalWeightsPerCluster = np.sum(pointProbsK, axis=1)
    clusterProbs = totalWeightsPerCluster / n

    means = np.zeros((k, d))
# to calculate new means of the clusters sum all the possibilities of cluster c
    for i in range(k):
        means[i] = np.dot(pointProbsK[i], img) / totalWeightsPerCluster[i]

    covariances = np.array([np.identity(3)*100] * k)
    for i in range(k):
        diff = img - means[i]
        covariances[i] = np.dot(pointProbsK[i] * diff.T, diff) /  totalWeightsPerCluster[i]
    
    return clusterProbs, means, covariances